<a href="https://colab.research.google.com/github/AndreiTsvirko/DataScience/blob/main/HW3/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Создание БД и таблицы

In [3]:
import psycopg2
from psycopg2 import sql
#Создаем БД в PG и подключимся к autompg_db
def create_database():
    try:
        # Подключаемся к стандартной базе данных для создания новой
        conn = psycopg2.connect(
            host="localhost",
            database="postgres",
            user="postgres",
            password="12345678"  # пароль
        )
        conn.autocommit = True
        cursor = conn.cursor()

        # Создаем новую базу данных
        cursor.execute("CREATE DATABASE autompg_db")
        print(" База данных autompg_db создана успешно!")

        cursor.close()
        conn.close()

    except Exception as e:
        print(f" Ошибка при создании базы данных: {e}")
#Создадим таблицу с разными типами данных
def create_table():
    try:
        # Подключаемся к созданной БД
        conn = psycopg2.connect(
            host="localhost",
            database="autompg_db",
            user="postgres",
            password="12345678"
        )
        cursor = conn.cursor()

        # SQL-запрос для создания таблицы с требуемыми типами данных
        create_table_query = """
        CREATE TABLE IF NOT EXISTS automobiles (
            id SERIAL PRIMARY KEY,                    -- Уникальный идентификатор
            mpg NUMERIC,                             -- Расход топлива (числовой)
            cylinders INTEGER,                       -- Количество цилиндров (целое)
            displacement NUMERIC,                    -- Объем двигателя (числовой)
            horsepower NUMERIC,                      -- Мощность (числовая)
            weight NUMERIC,                          -- Вес (числовой)
            acceleration NUMERIC,                    -- Ускорение (числовое)
            model_year INTEGER,                      -- Год модели (целое)
            origin TEXT,                             -- Страна происхождения (текст)
            car_name TEXT,                           -- Название автомобиля (текст)
            manufacture_date DATE                    -- Дата производства (дата)
        )
        """

        cursor.execute(create_table_query)
        conn.commit()
        print(" Таблица automobiles создана успешно.")

        cursor.close()
        conn.close()

    except Exception as e:
        print(f" Ошибка при создании таблицы: {e}")

#Имортируем данные

In [ ]:
import pandas as pd
import psycopg2
# Загрузим наш датасет auoMPG
def download_and_prepare_data():
    # URL датасета autoMPG
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"

    # Названия столбцов согласно документации датасета
    column_names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
                    'acceleration', 'model_year', 'origin', 'car_name']

    # Чтение данных из файла (разделитель - пробелы)
    df = pd.read_csv(url, delim_whitespace=True, names=column_names, na_values='?')

    # Добавляем столбец с датой производства (для демонстрации типа DATE)
    df['manufacture_date'] = pd.to_datetime(f'19{df["model_year"]}-01-01')

    # Преобразуем числовые коды стран в текстовые названия
    df['origin'] = df['origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

    # Удаляем строки с пропущенными значениями
    df = df.dropna()

    print(f"📊 Данные загружены: {df.shape[0]} строк, {df.shape[1]} столбцов")
    return df
# Вставляем выбранные данные из датасета в БД
def import_to_database(df):
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="autompg_db",
            user="postgres",
            password="12345678"
        )
        cursor = conn.cursor()

        # Вставляем данные построчно
        for index, row in df.iterrows():
            insert_query = """
            INSERT INTO automobiles
            (mpg, cylinders, displacement, horsepower, weight, acceleration,
             model_year, origin, car_name, manufacture_date)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """

            # Выполняем запрос с параметрами
            cursor.execute(insert_query, (
                row['mpg'], row['cylinders'], row['displacement'],
                row['horsepower'], row['weight'], row['acceleration'],
                row['model_year'], row['origin'], row['car_name'],
                row['manufacture_date']
            ))

        # Фиксируем изменения в базе данных
        conn.commit()
        print(f" Данные успешно импортированы в БД. ({len(df)} записей)")

        cursor.close()
        conn.close()

    except Exception as e:
        print(f" Ошибка при импорте данных: {e}")